In [2]:
import numpy as np
import pandas as pd
import torch
import os
import sys

/Users/carlschmidt/anaconda3/envs/bach/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torchvision import datasets, transforms
import torch.utils.data as data

In [40]:
df = pd.read_csv('data_analysis/val_imgs_df.csv')

In [80]:
df

,file_name,multiple_objects,background,color,brighter,darker,style,larger,smaller,object_blocking,...,partial_view,pattern,pose,shape,subcategory,texture,justification,one_word,label,str_label
0,ILSVRC2012_val_00004487.JPEG,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,close up of a pan fried sea bass.,sea bass close up,762,restaurant
1,ILSVRC2012_val_00003963.JPEG,0,1,1,0,0,1,0,0,0,...,0,0,1,0,0,0,sepia image of tiger,digitally altered,292,tiger
2,ILSVRC2012_val_00041992.JPEG,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,the bridge is brown,rare view,718,pier
3,ILSVRC2012_val_00028056.JPEG,0,1,0,0,0,0,0,0,0,...,0,1,1,1,0,0,the magnetic compass is on the bronze container,wood shape,635,magnetic compass
4,ILSVRC2012_val_00016832.JPEG,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,A dark image of a church.,dark,497,church
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48863,ILSVRC2012_val_00004973.JPEG,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,color is different,color,973,coral reef
48864,ILSVRC2012_val_00038750.JPEG,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,front view image of rocking chair made by digi...,window digital art,765,rocking chair
48865,ILSVRC2012_val_00043497.JPEG,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,right angle of analog clock with gold metallic...,right angle,409,analog clock
48866,ILSVRC2012_val_00005859.JPEG,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,A different design of a beer glass.,pattern background,441,beer glass


In [49]:
class MultiLabel(datasets.ImageFolder):
    def __init__(self, type_str, folder_path, annotations_path, *args, **kwargs):
        self.type = type_str
        super().__init__(folder_path + self.type, *args, **kwargs)
        if self.type == "val":
            self.label_df = pd.read_json(
                annotations_path + "imagenet_x_" + "train" + "_multi_factor.jsonl",
                lines=True,
            )
        if self.type == "train":
            self.label_df = pd.read_json(
                annotations_path + "imagenet_x_" + "val" + "_multi_factor.jsonl",
                lines=True,
            )

    # multilabel getitem method:
    def __getitem__(self, index: int):
        path, target = self.samples[index]
        sample = self.loader(path)

        if self.transform is not None:
            sample = self.transform(sample)

        # overwrite multilabel target:
        if self.type == "val":
            target = self.label_df[self.label_df.file_name == path[-20:]].values[0][2:18]

        if self.type == "train":
            target = self.label_df[self.label_df.file_name == path[-28:]].values[0][2:18]
        return sample, target

[0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1]


In [37]:
MultiLabel('train', )

[]


array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=object)

In [81]:
x, y = train_data.__getitem__(4)

In [82]:
y

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=object)

In [63]:
arr = df[df.file_name == x[-28:]].values[0][1:17]

In [6]:
path = 'data/annotations/'
dftrain_multi = pd.read_json(path + 'imagenet_x_train_multi_factor.jsonl', lines=True)
dfval_multi = pd.read_json(path + 'imagenet_x_val_multi_factor.jsonl', lines=True)

In [10]:
df_all_images_and_metalabels = pd.concat([dftrain_multi,dfval_multi])

In [19]:
df_all_images_and_metalabels.iloc[:,:18].drop('class',axis=1)

,file_name,multiple_objects,background,color,brighter,darker,style,larger,smaller,object_blocking,person_blocking,partial_view,pattern,pose,shape,subcategory,texture
0,n03633091_15528.JPEG,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,n02950826_6852.JPEG,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
2,n03404251_11352.JPEG,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0
3,n03930313_3341.JPEG,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,n02099267_827.JPEG,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48863,ILSVRC2012_val_00004973.JPEG,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0
48864,ILSVRC2012_val_00038750.JPEG,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0
48865,ILSVRC2012_val_00043497.JPEG,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0
48866,ILSVRC2012_val_00005859.JPEG,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0
